In [1]:
!pip install rasterio
import pandas as pd
import numpy as np
import os
import json
import requests
import rasterio
from rasterio.windows import Window
from shapely.geometry import Polygon
import numpy as np

In [2]:
def bounding_coordinates(place, country):
    url = f"https://nominatim.openstreetmap.org/search.php?q={place}+{country}&polygon_geojson=1&format=json"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        polygon_coordinates = 0
        bounding_coordinates = 0
        if(data[0]["class"] != "boundary"):
          for i in range(len(data)):
            if(data[i]["class"] == "boundary"):
              polygon_coordinates = data[i]['geojson']['coordinates'][0]
        else:
          polygon_coordinates = data[0]['geojson']['coordinates'][0]
        bounding_coordinates = [float(i) for i in data[0]['boundingbox']]
        return [polygon_coordinates, bounding_coordinates]

In [3]:
df = pd.read_excel("./Foodwatch_input_data.xlsx")
df

,Region,Country,IPC_Level,Crop_Cover_Percentage,Normalized_Food_Price,GDP_Per_Capita,Pests,Transport_Network_Density,Food_Storage_Groceries,Conflict_Per_Capita,Food_Wastage_Per_Capita
0,Bamingui,Central_African_Republic,3.0,1.727917,213.917728,538.0,17.981855,0.0,0.0,15.000000,103.0
1,Bangui,Central_African_Republic,2.0,5.000000,213.917728,538.0,17.981855,0.0,0.0,15.000000,103.0
2,Badakhshan,Afghanistan,3.0,2.948310,144.620601,422.0,0.436306,0.0,0.0,35.497200,82.0
3,Badghis,Afghanistan,3.0,11.321721,144.620601,422.0,0.436306,0.0,0.0,35.497200,82.0
4,Baghlan,Afghanistan,3.0,7.735584,144.620601,422.0,0.436306,0.0,0.0,35.497200,82.0
...,...,...,...,...,...,...,...,...,...,...,...
935,Chirumhanzu,Zimbabwe,3.0,7.169254,41924.478357,2088.0,8.389988,0.0,0.0,14.154411,100.0
936,Gweru,Zimbabwe,3.0,13.209877,41924.478357,2088.0,8.389988,0.0,0.0,14.154411,100.0
937,Kwekwe,Zimbabwe,3.0,7.103175,41924.478357,2088.0,8.389988,0.0,0.0,14.154411,100.0
938,Mberengwa,Zimbabwe,3.0,4.529617,41924.478357,2088.0,8.389988,0.0,0.0,14.154411,100.0


In [4]:
polygons = []
for i in range(df.shape[0]):
    row = df.loc[i, :]
    if i % 10 == 0:
        print(f"{i / 940}% complete")
    polygon = bounding_coordinates(row["Region"], row["Country"])[0]
    polygons.append(polygon)
len(polygons)

0.0% complete
0.010638297872340425% complete
0.02127659574468085% complete
0.031914893617021274% complete
0.0425531914893617% complete
0.05319148936170213% complete
0.06382978723404255% complete
0.07446808510638298% complete
0.0851063829787234% complete
0.09574468085106383% complete
0.10638297872340426% complete
0.11702127659574468% complete
0.1276595744680851% complete
0.13829787234042554% complete
0.14893617021276595% complete
0.1595744680851064% complete
0.1702127659574468% complete
0.18085106382978725% complete
0.19148936170212766% complete
0.20212765957446807% complete
0.2127659574468085% complete
0.22340425531914893% complete
0.23404255319148937% complete
0.24468085106382978% complete
0.2553191489361702% complete
0.26595744680851063% complete
0.2765957446808511% complete
0.2872340425531915% complete
0.2978723404255319% complete
0.30851063829787234% complete
0.3191489361702128% complete
0.32978723404255317% complete
0.3404255319148936% complete
0.35106382978723405% complete
0.3617

940

In [5]:
df["Polygon"] = polygons
df

,Region,Country,IPC_Level,Crop_Cover_Percentage,Normalized_Food_Price,GDP_Per_Capita,Pests,Transport_Network_Density,Food_Storage_Groceries,Conflict_Per_Capita,Food_Wastage_Per_Capita,Polygon
0,Bamingui,Central_African_Republic,3.0,1.727917,213.917728,538.0,17.981855,0.0,0.0,15.000000,103.0,"[[18.8606684, 8.8581526], [18.8634439, 8.85319..."
1,Bangui,Central_African_Republic,2.0,5.000000,213.917728,538.0,17.981855,0.0,0.0,15.000000,103.0,"[[18.5238866, 4.4176809], [18.5239499, 4.41750..."
2,Badakhshan,Afghanistan,3.0,2.948310,144.620601,422.0,0.436306,0.0,0.0,35.497200,82.0,"[[69.9801499, 37.4357056], [69.9938493, 37.415..."
3,Badghis,Afghanistan,3.0,11.321721,144.620601,422.0,0.436306,0.0,0.0,35.497200,82.0,"[[62.6655997, 34.9965666], [62.6743974, 34.977..."
4,Baghlan,Afghanistan,3.0,7.735584,144.620601,422.0,0.436306,0.0,0.0,35.497200,82.0,"[[68.0039979, 35.1380668], [68.0040743, 35.137..."
...,...,...,...,...,...,...,...,...,...,...,...,...
935,Chirumhanzu,Zimbabwe,3.0,7.169254,41924.478357,2088.0,8.389988,0.0,0.0,14.154411,100.0,"[[30.0565486, -19.2069222], [30.0668047, -19.2..."
936,Gweru,Zimbabwe,3.0,13.209877,41924.478357,2088.0,8.389988,0.0,0.0,14.154411,100.0,"[[29.0936508, -19.3762663], [29.1010736, -19.3..."
937,Kwekwe,Zimbabwe,3.0,7.103175,41924.478357,2088.0,8.389988,0.0,0.0,14.154411,100.0,"[[28.877778, -18.7712776], [28.8905452, -18.82..."
938,Mberengwa,Zimbabwe,3.0,4.529617,41924.478357,2088.0,8.389988,0.0,0.0,14.154411,100.0,"[[29.4623309, -21.0237186], [29.4717976, -21.0..."


In [16]:
df.to_json("./polygon_data.json", orient="records")

In [2]:
df = pd.read_json("./polygon_data.json")
df

,Region,Country,IPC_Level,Crop_Cover_Percentage,Normalized_Food_Price,GDP_Per_Capita,Pests,Transport_Network_Density,Food_Storage_Groceries,Conflict_Per_Capita,Food_Wastage_Per_Capita,Polygon
0,Bamingui,Central_African_Republic,3,1.727917,213.917728,538,17.981855,0,0,15.000000,103,"[[18.8606684, 8.8581526], [18.8634439, 8.85319..."
1,Bangui,Central_African_Republic,2,5.000000,213.917728,538,17.981855,0,0,15.000000,103,"[[18.5238866, 4.4176809], [18.5239499, 4.41750..."
2,Badakhshan,Afghanistan,3,2.948310,144.620601,422,0.436306,0,0,35.497200,82,"[[69.9801499, 37.4357056], [69.9938493, 37.415..."
3,Badghis,Afghanistan,3,11.321721,144.620601,422,0.436306,0,0,35.497200,82,"[[62.6655997, 34.9965666], [62.6743974, 34.977..."
4,Baghlan,Afghanistan,3,7.735584,144.620601,422,0.436306,0,0,35.497200,82,"[[68.0039979, 35.1380668], [68.0040743, 35.137..."
...,...,...,...,...,...,...,...,...,...,...,...,...
935,Chirumhanzu,Zimbabwe,3,7.169254,41924.478357,2088,8.389988,0,0,14.154411,100,"[[30.0565486, -19.2069222], [30.0668047, -19.2..."
936,Gweru,Zimbabwe,3,13.209877,41924.478357,2088,8.389988,0,0,14.154411,100,"[[29.0936508, -19.3762663], [29.1010736, -19.3..."
937,Kwekwe,Zimbabwe,3,7.103175,41924.478357,2088,8.389988,0,0,14.154411,100,"[[28.877778, -18.7712776], [28.8905452, -18.82..."
938,Mberengwa,Zimbabwe,3,4.529617,41924.478357,2088,8.389988,0,0,14.154411,100,"[[29.4623309, -21.0237186], [29.4717976, -21.0..."


In [4]:
df[df["Region"] == 'Zambezi']

,Region,Country,IPC_Level,Crop_Cover_Percentage,Normalized_Food_Price,GDP_Per_Capita,Pests,Transport_Network_Density,Food_Storage_Groceries,Conflict_Per_Capita,Food_Wastage_Per_Capita,Polygon
467,Zambezi,Namibia,3,0.953437,169.082940,4745,2.531474,0,0,39.34553,92,"[[22.5000204, -18.1175994], [22.629244, -18.09..."
861,Zambezi,Zambia,3,0.000000,270.774527,1413,11.956836,0,0,3.29670,78,"[[22.0003946, -13.7127252], [22.0005241, -13.7..."


In [5]:
df.iloc[0, :]

Region                                                                Bamingui
Country                                               Central_African_Republic
IPC_Level                                                                    3
Crop_Cover_Percentage                                                 1.727917
Normalized_Food_Price                                               213.917728
GDP_Per_Capita                                                             538
Pests                                                                17.981855
Transport_Network_Density                                                    0
Food_Storage_Groceries                                                       0
Conflict_Per_Capita                                                       15.0
Food_Wastage_Per_Capita                                                    103
Polygon                      [[18.8606684, 8.8581526], [18.8634439, 8.85319...
Name: 0, dtype: object

In [12]:
polygons = []
for i in range(df.shape[0]):
    row = df.loc[i, :]
    if i % 10 == 0:
        print(f"{i / 940}% complete")
    newPolygon = []
    for coord in row["Polygon"]:
        newPolygon.append([coord[1], coord[0]])
    polygons.append(newPolygon)
len(polygons)

0.0% complete
0.010638297872340425% complete
0.02127659574468085% complete
0.031914893617021274% complete
0.0425531914893617% complete
0.05319148936170213% complete
0.06382978723404255% complete
0.07446808510638298% complete
0.0851063829787234% complete
0.09574468085106383% complete
0.10638297872340426% complete
0.11702127659574468% complete
0.1276595744680851% complete
0.13829787234042554% complete
0.14893617021276595% complete
0.1595744680851064% complete
0.1702127659574468% complete
0.18085106382978725% complete
0.19148936170212766% complete
0.20212765957446807% complete
0.2127659574468085% complete
0.22340425531914893% complete
0.23404255319148937% complete
0.24468085106382978% complete
0.2553191489361702% complete
0.26595744680851063% complete
0.2765957446808511% complete
0.2872340425531915% complete
0.2978723404255319% complete
0.30851063829787234% complete
0.3191489361702128% complete
0.32978723404255317% complete
0.3404255319148936% complete
0.35106382978723405% complete
0.3617

940

In [14]:
df["Polygon"] = polygons
df

,Region,Country,IPC_Level,Crop_Cover_Percentage,Normalized_Food_Price,GDP_Per_Capita,Pests,Transport_Network_Density,Food_Storage_Groceries,Conflict_Per_Capita,Food_Wastage_Per_Capita,Polygon
0,Bamingui,Central_African_Republic,3,1.727917,213.917728,538,17.981855,0,0,15.000000,103,"[[8.8581526, 18.8606684], [8.8531986, 18.86344..."
1,Bangui,Central_African_Republic,2,5.000000,213.917728,538,17.981855,0,0,15.000000,103,"[[4.4176809, 18.5238866], [4.4175039, 18.52394..."
2,Badakhshan,Afghanistan,3,2.948310,144.620601,422,0.436306,0,0,35.497200,82,"[[37.4357056, 69.9801499], [37.4154638, 69.993..."
3,Badghis,Afghanistan,3,11.321721,144.620601,422,0.436306,0,0,35.497200,82,"[[34.9965666, 62.6655997], [34.9779739, 62.674..."
4,Baghlan,Afghanistan,3,7.735584,144.620601,422,0.436306,0,0,35.497200,82,"[[35.1380668, 68.0039979], [35.137505, 68.0040..."
...,...,...,...,...,...,...,...,...,...,...,...,...
935,Chirumhanzu,Zimbabwe,3,7.169254,41924.478357,2088,8.389988,0,0,14.154411,100,"[[-19.2069222, 30.0565486], [-19.2168177, 30.0..."
936,Gweru,Zimbabwe,3,13.209877,41924.478357,2088,8.389988,0,0,14.154411,100,"[[-19.3762663, 29.0936508], [-19.3790015, 29.1..."
937,Kwekwe,Zimbabwe,3,7.103175,41924.478357,2088,8.389988,0,0,14.154411,100,"[[-18.7712776, 28.877778], [-18.8228629, 28.89..."
938,Mberengwa,Zimbabwe,3,4.529617,41924.478357,2088,8.389988,0,0,14.154411,100,"[[-21.0237186, 29.4623309], [-21.0288735, 29.4..."


In [15]:
df.to_json("./polygon_data.json")